Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
___
В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). 

**Цель:** построить модель для задачи классификации, которая выберет подходящий тариф.

Построим модель с максимально большим значением accuracy. 

Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до *0.75*. 

Проверим accuracy на тестовой выборке самостоятельно.

## 1. Откроем файл с данными и изучим его

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [2]:
#data = pd.read_csv('datasets/users_behavior.csv')
data = pd.read_csv('/datasets/users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Датасет состоит из 3214 объектов и 5 признаков. 

Целевой признак – `is_ultra`. Он является категориальным, следовательно, перед нами задача классификации.

Модель машинного обучения будет предсказывать значение 1, если клиенту нужно предложить тариф "Ультра" или 0, если тариф "Смарт".

## 2. Разделим исходные данные на обучающую/валидационную/тестовую выборки

Будем делить в следующем соотношении:

- 60% - обучающая выборка
- 20% - валидационная выборка
- 20% - тестовая выборка

Так же зафиксируем random_state = 12345

In [4]:
rnd = 1234

data_train, data_valid = train_test_split(data, test_size=0.4, random_state=rnd)
data_valid, data_test = train_test_split(data_valid, test_size=0.5, random_state=rnd)

На обучающей выборке обучаем, на валидационной проверяем, а на тестовой оцениваем модель.

## 3. Исследуем разные модели

Подготовка признаков и таргета для обучающей и валидационной и тестовой выборок

In [5]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

Функция для расчёта accuracy модели на валидационной выборке:

In [6]:
def accuracy_score_valid(model):
    predictions_valid = model.predict(features_valid)
    return accuracy_score(target_valid, predictions_valid)

### Дерево решений
Для дерева решений подберем гиперпараметр max_depth (глубина) на кросс валидации.

In [7]:
for max_depth in range(1, 21, 2):
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=rnd)
    model.fit(features_train, target_train)
    print(f"max_depth = {max_depth}:\t{accuracy_score_valid(model)}")

max_depth = 1:	0.776049766718507
max_depth = 3:	0.8118195956454122
max_depth = 5:	0.8087091757387247
max_depth = 7:	0.8320373250388803
max_depth = 9:	0.8180404354587869
max_depth = 11:	0.8180404354587869
max_depth = 13:	0.8040435458786936
max_depth = 15:	0.7744945567651633
max_depth = 17:	0.7480559875583204
max_depth = 19:	0.744945567651633


По результатам на валидационной выборке, дерево решений имеет самую высокую оценку правильности приблизительно 0.832, когда гиперпараметр глубины равен 7.

### Случайный лес
Подберем количество деревьев n_estimators  при  max_depth=7.

Количества деревьев, на которых испытываем модель – от 10 до 100 с шагом 10.

In [8]:
for estim in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators=estim, max_depth=7, random_state=rnd)
    model.fit(features_train, target_train)
    print(f"n_estimators = {estim}:\t{accuracy_score_valid(model)}")

n_estimators = 10:	0.8227060653188181
n_estimators = 20:	0.8258164852255054
n_estimators = 30:	0.8289269051321928
n_estimators = 40:	0.8320373250388803
n_estimators = 50:	0.8351477449455676
n_estimators = 60:	0.833592534992224
n_estimators = 70:	0.833592534992224
n_estimators = 80:	0.8367029548989113
n_estimators = 90:	0.833592534992224
n_estimators = 100:	0.8351477449455676


Модель случайного леса имеет более высокую accuracy, но не на много – 0.837 при количестве деревьев леса n_estimators=80.

### Логистическая регрессия

In [9]:
def logistic_regression(solver):
    model = LogisticRegression(solver=solver, random_state=rnd)
    model.fit(features_train, target_train)
    return accuracy_score_valid(model)

In [10]:
logistic_regression('liblinear')

0.7216174183514774

Логистическая регрессия на валидационной выборке показала наихудшие результаты. 

Попробуем изменить гиперпараметры, например, solver – алгоритм задачи оптимизации.

In [11]:
print('lbfgs:', logistic_regression('lbfgs'))

lbfgs: 0.7589424572317263


При использовании lbfgs-солвера хоть и удалось достичь лучшего решения относительно liblinear-солвера, но это решение не лучше, чем у дерева или случайного леса.

## 4. Проверка модели на тестовой выборке

В результате эксперимента на валидационной выборке в части 3 было определено, что самую высокую оценку правильности дает лес решений с глубиной  max_depth=7 и количеством деревьев n_estimator=80.

Проверим модель на тестовой выборке.

In [12]:
model = RandomForestClassifier(n_estimators=80, max_depth=7, random_state=rnd)
model.fit(features_train, target_train)

predictions_test = model.predict(features_test)
accuracy_score(target_test, predictions_test)

0.8087091757387247

На тестовой выборке данная модель показывает приближенное к валидационной выборке значение accuracy. 

Т.е. здесь нет проблемы с переобучением, следовательно, нам удалось построить модель для задачи бинарной классификации для выбора оптимального тарифа.

## 5. Доп. задание: проверка модели на адекватность

Необходимо, чтобы модель машинного обучения хорого работала на практике, в т.ч. не испытывала проблем с переообучением.

Проверка модели машинного обучения на адекватность может иметь разные подходы. 

Например, проверка правильности выбора метрики. В задачах классификации мы используем accuracy, но в случае дисбаланса классов, т.е. когда в целевом признаке 0 встречается чаще, чем 1 (или наоборот), на accuracy нельзя полагаться.

Проверим, как распределены классы в нашей задаче:

In [13]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Для задач бинарной классификации могут быть использованы следующие методы:
- Матрица ошибок / confusion matrix
- Тесты бинарной классификации / binary classification tests
- Коэффициент конверсии / conversion rates
- ROC-кривая / ROC curve
- Совокупный доход / cumulative gain
- Lift-кривая / lift chart

In [14]:
data_test['is_ultra'].value_counts()

0    449
1    194
Name: is_ultra, dtype: int64

Данные распределены неравномерно, как в датасете в целом, так и в тестовой выборке: `is_ultra` с меткой 0 перевешивает в 2 раза.

In [15]:
(data_test['is_ultra']==0).sum() / data_test.shape[0]

0.6982892690513219

Доля бОльшего класса тестовой выборки равна ~0.7; полученная нами модель имеет accuracy ~0.81. 

Таким образом, можно считать модель адекватной для использования на практике.

Стоит отметить, что обученная модель должна быть учитывать предположения, что дисбаланс классов в обучающей выборке будет таким же и на тестовой выборке.